In [10]:
import mysql.connector
import pandas as pd

# --- DB connection ---
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ssm",
    database="hospital"
)
cursor = conn.cursor()

# 1) Get up to 50 unique patients
cursor.execute("""
    SELECT DISTINCT patient_id
    FROM HealthRecords
    ORDER BY patient_id
    LIMIT 50
""")
patient_ids = [r[0] for r in cursor.fetchall()]

if not patient_ids:
    print("No patients found in HealthRecords.")
    conn.close()
    raise SystemExit

# 2) For those patients, get their *latest* record
fmt = ",".join(["%s"] * len(patient_ids))
cursor.execute(f"""
    SELECT hr.patient_id,
           hr.blood_pressure_systolic AS systolic,
           hr.blood_pressure_diastolic AS diastolic,
           hr.heart_rate,
           hr.blood_sugar_level AS sugar,
           hr.BMI AS bmi,
           hr.date_recorded
    FROM HealthRecords hr
    JOIN (
        SELECT patient_id, MAX(date_recorded) AS max_date
        FROM HealthRecords
        WHERE patient_id IN ({fmt})
        GROUP BY patient_id
    ) last ON last.patient_id = hr.patient_id AND last.max_date = hr.date_recorded
    ORDER BY hr.patient_id
""", tuple(patient_ids))

rows = cursor.fetchall()
conn.close()

# 3) To DataFrame + make numbers safe (handle NULL/strings)
df = pd.DataFrame(rows, columns=[
    "patient_id", "systolic", "diastolic", "heart_rate", "sugar", "bmi", "date_recorded"
])

for col in ["systolic", "diastolic", "heart_rate", "sugar", "bmi"]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# 4) Derive conditions & meds (one row per patient)
conditions_list = []
for _, r in df.iterrows():
    conds, meds = [], []

    if r["sugar"] > 140:
        conds.append("Diabetes")
        meds.append("Metformin")
    if r["systolic"] > 140 or r["diastolic"] > 90:
        conds.append("Hypertension")
        meds.append("Amlodipine")
    if r["bmi"] > 30:
        conds.append("Obesity")
        meds.append("Lifestyle modification")
    if r["heart_rate"] > 100:
        conds.append("Tachycardia")
        meds.append("Beta-blocker")

    if not conds:
        conds.append("Normal")
        meds.append("None")

    conditions_list.append({
        "patient_id": int(r["patient_id"]),
        "previous_medical_condition": ", ".join(conds),
        "medications_used": ", ".join(meds)
    })

# Ensure exactly one row per patient (in case of edge dupes)
out = (pd.DataFrame(conditions_list)
         .drop_duplicates(subset="patient_id")
         .sort_values("patient_id")
         .head(50))

# 5) Save CSV
out.to_csv("medical_history_50patients.csv", index=False)
print(f"✅ Wrote {len(out)} rows to medical_history_50patients.csv")







✅ Wrote 50 rows to medical_history_50patients.csv
